#Implementing Multi-CNN Model Using GoogleNet Architecture

### Importing all necessary libraries

In [0]:
import numpy as np 
import pandas as pd 
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
import warnings
warnings.filterwarnings('ignore')

###Connecting to my Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


###Unzipping the UTKFace Zip file from Google drive

In [0]:
!unzip "/content/drive/My Drive/UTKFace/utkface-new.zip"

In [0]:
import glob
total_images=0
utkface = []
utkface +=  glob.glob('/content/UTKFace/*')
total_images = len(utkface)
print(total_images)

23708


###Details of UTKFace

#### Age(0-116), Gender(Male,Female) and Race(White,Black,Asian,Indian and Others)

In [0]:
IMG_WIDTH = IMG_HEIGHT = 200
Id_Gen = {0: 'male', 1: 'female'}
Gen_Id = dict((g, i) for i, g in Id_Gen.items())
Id_Race = {0: 'white', 1: 'black', 2: 'asian', 3: 'indian', 4: 'others'}
Race_Id = dict((r, i) for i, r in Id_Race.items())

Id_Gen, Gen_Id, Id_Race, Race_Id

({0: 'male', 1: 'female'},
 {'female': 1, 'male': 0},
 {0: 'white', 1: 'black', 2: 'asian', 3: 'indian', 4: 'others'},
 {'asian': 2, 'black': 1, 'indian': 3, 'others': 4, 'white': 0})

In [0]:
len(utkface)

23708

###Collecting details from Each Image

In [0]:
details=[]
for img in utkface:
  try:
    age, gender, race, _ = img.split("_")
    details.append([int(age[17:]), Id_Gen[int(gender)], Id_Race[int(race)]])
  except:
    details.append([None,None,None])

len(details),len(utkface)

(23708, 23708)

###Converting to a DataFrame

In [0]:
data = pd.DataFrame(details)
data['images'] = utkface
data.columns = ['age', 'gender', 'race', 'images']
data.head()

,age,gender,race,images
0,1.0,male,asian,/content/UTKFace/1_0_2_20161220144911423.jpg.c...
1,38.0,male,black,/content/UTKFace/38_0_1_20170116002845718.jpg....
2,36.0,female,black,/content/UTKFace/36_1_1_20170112231520246.jpg....
3,16.0,female,black,/content/UTKFace/16_1_1_20170109214212884.jpg....
4,25.0,female,white,/content/UTKFace/25_1_0_20170116205838424.jpg....


###Droping rows that contains Null/NaN values

In [0]:
data.isnull().sum()

age       3
gender    3
race      3
images    0
dtype: int64

In [0]:
data = data.dropna()

In [0]:
data.isnull().sum(),len(data)

(age       0
 gender    0
 race      0
 images    0
 dtype: int64, 23705)

###Spliting the data 

In [0]:
p = np.random.permutation(len(data))
train_up_to = int(len(data) * 0.7)
train_idx = p[:train_up_to]
test_idx = p[train_up_to:]

# split train_idx further into training and validation set
train_up_to = int(train_up_to * 0.7)
train_idx, valid_idx = train_idx[:train_up_to], train_idx[train_up_to:]

data['gender_id'] = data['gender'].map(lambda gender: Gen_Id[gender])
data['race_id'] = data['race'].map(lambda race: Race_Id[race])

max_age = data['age'].max()
len(train_idx), len(valid_idx), len(test_idx), max_age

(11615, 4978, 7112, 116.0)

###Image Preprocessing

In [0]:
from PIL import Image
from keras.utils import to_categorical

def Img_preprocesser(data, indexes, train, batch_size=16):
    images, ages, races, genders = [], [], [], []
    while True:
        for i in indexes:
            file_ = data.iloc[i]
            image, age, race, gender = file_['images'], file_['age'], file_['race_id'], file_['gender_id']
            img = Image.open(image)
            img = img.resize((IMG_WIDTH, IMG_HEIGHT))
            img = np.array(img) / 255.0
            images.append(img)
            ages.append(age / max_age)
            races.append(to_categorical(race, len(Race_Id)))
            genders.append(to_categorical(gender, 2))
            if len(images) >= batch_size:
                yield np.array(images), [np.array(ages), np.array(races), np.array(genders)]
                images, ages, races, genders = [], [], [], []
        if not train:
            break

Using TensorFlow backend.


###Importing Libraries

Libraries that are necessary to implement GoogleNet Inception Architecture

In [0]:
import keras,os
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,Dropout, Dense, Input, concatenate,GlobalAveragePooling2D, AveragePooling2D,Flatten,GlobalMaxPool2D

from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf

from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler

###Inception Model

Inception Function 

In [0]:
def inception_module(x,filters_1x1,filters_3x3_reduce,filters_3x3,filters_5x5_reduce,filters_5x5,filters_pool_proj,name=None):
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output


###Layers of GooleNet Architecture

Implementing GoogleNet Architecture and converting the Architecture to Multi-CNN

In [0]:
input_layer = Input(shape=(200, 200, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)


x = inception_module(x,filters_1x1=64,filters_3x3_reduce=96,filters_3x3=128,filters_5x5_reduce=16,filters_5x5=32,filters_pool_proj=32,name='inception_3a')

x = inception_module(x,filters_1x1=128,filters_3x3_reduce=128,filters_3x3=192,filters_5x5_reduce=32,filters_5x5=96,filters_pool_proj=64,name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,filters_1x1=192,filters_3x3_reduce=96,filters_3x3=208,filters_5x5_reduce=16,filters_5x5=48,filters_pool_proj=64,name='inception_4a')


x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(10, activation='softmax', name='auxilliary_output_1')(x1)


x = inception_module(x,filters_1x1=160,filters_3x3_reduce=112,filters_3x3=224,filters_5x5_reduce=24,filters_5x5=64,filters_pool_proj=64,name='inception_4b')

x = inception_module(x,filters_1x1=128,filters_3x3_reduce=128,filters_3x3=256,filters_5x5_reduce=24,filters_5x5=64,filters_pool_proj=64,name='inception_4c')

x = inception_module(x,filters_1x1=112,filters_3x3_reduce=144,filters_3x3=288,filters_5x5_reduce=32,filters_5x5=64,filters_pool_proj=64,name='inception_4d')

x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(10, activation='softmax', name='auxilliary_output_2')(x2)


x = inception_module(x,filters_1x1=256,filters_3x3_reduce=160,filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,filters_pool_proj=128,name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,filters_1x1=256,filters_3x3_reduce=160,filters_3x3=320,filters_5x5_reduce=32,filters_5x5=128,filters_pool_proj=128,name='inception_5a')

x = inception_module(x,filters_1x1=384,filters_3x3_reduce=192,filters_3x3=384,filters_5x5_reduce=48,filters_5x5=128,filters_pool_proj=128,name='inception_5b')


x = Dropout(0.4)(x)

#Expanding GoogleNet to achieve Multi-CNN Model

bottleneck = GlobalMaxPool2D()(x)
 
# for age calculation
x = Dense(units=128, activation='relu')(bottleneck)
age_output = Dense(units=1, activation='sigmoid', name='age_output')(x)
 
# for race prediction
x = Dense(units=128, activation='relu')(bottleneck)
race_output = Dense(units=len(Race_Id), activation='softmax', name='race_output')(x)
 
# for gender prediction
x = Dense(units=128, activation='relu')(bottleneck)
gender_output = Dense(units=len(Gen_Id), activation='softmax', name='gender_output')(x)
 



In [0]:
model = Model(inputs=input_layer, outputs=[age_output, race_output, gender_output])
adam = keras.optimizers.Adam(0.001)
model.compile(optimizer=adam, 
              loss={'age_output': 'mse', 'race_output': 'categorical_crossentropy', 'gender_output': 'categorical_crossentropy'},
              loss_weights={'age_output': 2., 'race_output': 1.5, 'gender_output': 1.},
              metrics={'age_output': 'mae', 'race_output': 'accuracy', 'gender_output': 'accuracy'})


###Model Summary

In [0]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 100, 100, 64) 9472        input_4[0][0]                    
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 50, 50, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 50, 50, 64)   4160        max_pool_1_3x3/2[0][0]           
____________________________________________________________________________________________

###Fitting the Model

In [0]:
from keras.callbacks import ModelCheckpoint

batch_size = 256
valid_batch_size = 256
train_ = Img_preprocesser(data, train_idx, train=True, batch_size=batch_size)
valid_ = Img_preprocesser(data, valid_idx, train=True, batch_size=valid_batch_size)

callbacks = [ModelCheckpoint("./model_checkpoint", monitor='val_loss')]

# if torch.cuda.is_available():
#     device = torch.device("cuda")         
#     train_ = torch.from_numpy(train_)
#     valid_ = torch.from_numpy(valid_)


history = model.fit_generator(train_,steps_per_epoch=len(train_idx)//batch_size,epochs=75,callbacks=callbacks,validation_data=valid_,validation_steps=len(valid_idx)//valid_batch_size)



Epoch 1/75
45/45 [==============================] - 111s 2s/step - loss: 3.7845 - age_output_loss: 0.0941 - race_output_loss: 1.7709 - gender_output_loss: 0.9400 - age_output_mean_absolute_error: 0.2546 - race_output_acc: 0.3938 - gender_output_acc: 0.4995 - val_loss: 2.9694 - val_age_output_loss: 0.0286 - val_race_output_loss: 1.4769 - val_gender_output_loss: 0.6968 - val_age_output_mean_absolute_error: 0.1295 - val_race_output_acc: 0.4297 - val_gender_output_acc: 0.4729
Epoch 2/75
45/45 [==============================] - 88s 2s/step - loss: 2.9002 - age_output_loss: 0.0315 - race_output_loss: 1.4343 - gender_output_loss: 0.6857 - age_output_mean_absolute_error: 0.1373 - race_output_acc: 0.4341 - gender_output_acc: 0.5525 - val_loss: 2.8758 - val_age_output_loss: 0.0371 - val_race_output_loss: 1.4177 - val_gender_output_loss: 0.6752 - val_age_output_mean_absolute_error: 0.1651 - val_race_output_acc: 0.4679 - val_gender_output_acc: 0.5997
Epoch 3/75
45/45 [=============================

###Testing the Model

In [0]:
test_= Img_preprocesser(data, test_idx, train=False, batch_size=128)
dict(zip(model.metrics_names, model.evaluate_generator(test_, steps=len(test_idx)//128)))

{'age_output_loss': 0.015469023382121866,
 'age_output_mean_absolute_error': 0.09864615879275582,
 'gender_output_acc': 0.8650568181818182,
 'gender_output_loss': 0.5152439732443203,
 'loss': 2.4811687317761506,
 'race_output_acc': 0.7303977272727272,
 'race_output_loss': 1.2899911544539713}

###Predicting

In [0]:
test_ = Img_preprocesser(data, test_idx, train=False, batch_size=128)
x_test, (age_true, race_true, gender_true)= next(test_)
age_pred, race_pred, gender_pred = model.predict_on_batch(x_test)

###Saving the Model Using Keras

In [0]:
model.save("Age_Gender_Race_Model_1.h5")

In [0]:
from keras.models import load_model

###Loading the Saved Model

In [0]:
model = load_model('/content/Age_Gender_Race_Model_1.h5')

model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 100, 100, 64) 9472        input_4[0][0]                    
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 50, 50, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 50, 50, 64)   4160        max_pool_1_3x3/2[0][0]           
____________________________________________________________________________________________

###Testing Sample Image

In [0]:
test_img = Image.open('/content/download.jpg')

In [0]:

img = test_img.resize((IMG_WIDTH, IMG_HEIGHT))
img = np.array(img) / 255.0

In [0]:
img = img.reshape(-1,200,200,3)

In [0]:
age, race, gender = model.predict(img)

In [0]:
print("Age:",int(age[0]*100),"Race:" ,Id_Race[race.argmax()], "Gender:",Id_Gen[gender.argmax()])

Age: 20 Race: white Gender: male
